# SWE-Bench 自定义沙箱工具完整演示

本 Cookbook 演示如何:
1. 创建带 envd 挂载的 SWE-Bench 沙箱工具
2. 使用 e2b 接口启动和管理沙箱实例
3. 在沙箱中执行命令和上传文件

**镜像配置**：

- **基础镜像**: `ccr.ccs.tencentyun.com/pengdrumli/slimshetty-swebench-lite:sweb.eval.x86_64.astropy__astropy-12907`
- **挂载镜像**: `ccr.ccs.tencentyun.com/archerlliu/envd:20260115_201017`

安装依赖

In [12]:
%pip install 'tencentcloud-sdk-python>=3.1.32' e2b_code_interpreter

import tencentcloud
print(f"✅ SDK版本: {tencentcloud.__version__}")


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ SDK版本: 3.1.42


In [59]:
import os
import json
import uuid
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.common.exception.tencent_cloud_sdk_exception import TencentCloudSDKException
from tencentcloud.ags.v20250920 import ags_client, models
from e2b_code_interpreter import Sandbox

print("✅ 所有模块导入成功")

✅ 所有模块导入成功


环境配置与初始化

In [ ]:
# 配置腾讯云凭证
if not os.getenv("TENCENTCLOUD_SECRET_ID"):
    os.environ["TENCENTCLOUD_SECRET_ID"] = "xxx"
if not os.getenv("TENCENTCLOUD_SECRET_KEY"):
    os.environ["TENCENTCLOUD_SECRET_KEY"] = "xxx"
if not os.getenv("TENCENTCLOUD_REGION"):
    os.environ["TENCENTCLOUD_REGION"] = "ap-guangzhou"

# 创建 AGS 客户端
cred = credential.Credential(
    os.environ.get("TENCENTCLOUD_SECRET_ID"),
    os.environ.get("TENCENTCLOUD_SECRET_KEY")
)

http_profile = HttpProfile()
http_profile.endpoint = "ags.tencentcloudapi.com"

client_profile = ClientProfile()
client_profile.httpProfile = http_profile

client = ags_client.AgsClient(cred, os.getenv("TENCENTCLOUD_REGION"), client_profile)

print("✅ AGS客户端创建成功")
print(f"📍 地域: {os.getenv('TENCENTCLOUD_REGION')}")
print(f"{os.environ.get("TENCENTCLOUD_SECRET_ID")}")
print(f"{os.environ.get("TENCENTCLOUD_SECRET_KEY")}")

创建自定义沙箱工具

In [61]:
# SWE-Bench 沙箱配置
TOOL_NAME = "swe-bench-astropy-12907"
BASE_IMAGE = "ccr.ccs.tencentyun.com/pengdrumli/slimshetty-swebench-lite:sweb.eval.x86_64.astropy__astropy-12907"
MOUNT_IMAGE = "ccr.ccs.tencentyun.com/archerlliu/envd:20260115_201017"
ROLE_ARN = "qcs::cam::uin/3321337994:roleName/tcr-full-ags"

print(f"工具名称: {TOOL_NAME}")
print(f"基础镜像: {BASE_IMAGE}")
print(f"挂载镜像: {MOUNT_IMAGE}")

工具名称: swe-bench-astropy-12907
基础镜像: ccr.ccs.tencentyun.com/pengdrumli/slimshetty-swebench-lite:sweb.eval.x86_64.astropy__astropy-12907
挂载镜像: ccr.ccs.tencentyun.com/archerlliu/envd:20260115_201017


In [62]:
def create_swe_bench_tool():
    """创建 SWE-Bench 沙箱工具"""
    try:
        req = models.CreateSandboxToolRequest()
        req.ToolName = TOOL_NAME
        req.ToolType = "custom"
        req.Description = "SWE-Bench environment with envd"
        req.DefaultTimeout = "15m"
        req.RoleArn = ROLE_ARN
        req.ClientToken = str(uuid.uuid4())
        
        # 网络配置
        req.NetworkConfiguration = models.NetworkConfiguration()
        req.NetworkConfiguration.NetworkMode = "PUBLIC"
        
        # 自定义配置
        req.CustomConfiguration = models.CustomConfiguration()
        req.CustomConfiguration.Image = BASE_IMAGE
        req.CustomConfiguration.ImageRegistryType = "personal"
        req.CustomConfiguration.Command = ["/bin/bash", "-c"]
        req.CustomConfiguration.Args = ["/mnt/envd -port 49983"]
        
        # 环境变量
        env_vars = []
        env_var = models.EnvVar()
        env_var.Name = "LANG"
        env_var.Value = "en_US.UTF-8"
        env_vars.append(env_var)
        
        env_var = models.EnvVar()
        env_var.Name = "PATH"
        env_var.Value = "/envd/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
        env_vars.append(env_var)
        
        env_var = models.EnvVar()
        env_var.Name = "DEBIAN_FRONTEND"
        env_var.Value = "noninteractive"
        env_vars.append(env_var)
        
        env_var = models.EnvVar()
        env_var.Name = "PYTHONUNBUFFERED"
        env_var.Value = "1"
        env_vars.append(env_var)
        
        req.CustomConfiguration.Env = env_vars
        # 资源配置
        req.CustomConfiguration.Resources = models.ResourceConfiguration()
        req.CustomConfiguration.Resources.CPU = "4"
        req.CustomConfiguration.Resources.Memory = "8Gi"
        
        # 探针配置
        req.CustomConfiguration.Probe = models.ProbeConfiguration()
        req.CustomConfiguration.Probe.HttpGet = models.HttpGetAction()
        req.CustomConfiguration.Probe.HttpGet.Path = "/health"
        req.CustomConfiguration.Probe.HttpGet.Port = 49983
        req.CustomConfiguration.Probe.HttpGet.Scheme = "HTTP"
        req.CustomConfiguration.Probe.ReadyTimeoutMs = 30000
        req.CustomConfiguration.Probe.ProbeTimeoutMs = 1000
        req.CustomConfiguration.Probe.ProbePeriodMs = 2000
        req.CustomConfiguration.Probe.SuccessThreshold = 1
        req.CustomConfiguration.Probe.FailureThreshold = 30
        
        # envd 存储挂载
        storage_mount = models.StorageMount()
        storage_mount.Name = "envd-storage"
        storage_mount.MountPath = "/mnt/envd"
        storage_mount.ReadOnly = True
        storage_mount.StorageSource = models.StorageSource()
        storage_mount.StorageSource.Image = models.ImageStorageSource()
        storage_mount.StorageSource.Image.Reference = MOUNT_IMAGE
        storage_mount.StorageSource.Image.ImageRegistryType = "personal"
        storage_mount.StorageSource.Image.SubPath = "/usr/bin/envd"
        req.StorageMounts = [storage_mount]
        
        # 创建工具
        print(f"🔧 正在创建工具: {TOOL_NAME}")
        resp = client.CreateSandboxTool(req)
        print(f"✅ 工具创建成功")
        print(f"🆔 Tool ID: {resp.ToolId}")
        return resp.ToolId
    except TencentCloudSDKException as err:
        print(f"❌ 创建失败: {err}")
        if "already exists" in str(err):
            print("ℹ️  工具已存在，可以直接使用")
        return None

print("✅ 工具创建函数定义完成")

✅ 工具创建函数定义完成


In [63]:
# 创建工具（如果已存在会提示）
tool_id = create_swe_bench_tool()

🔧 正在创建工具: swe-bench-astropy-12907
✅ 工具创建成功
🆔 Tool ID: sdt-1fitksfj


In [64]:
# # 查询工具列表
# try:
#     req = models.DescribeSandboxToolListRequest()
#     req.Limit = 10
#     resp = client.DescribeSandboxToolList(req)
#     print(f"📋 查询到 {resp.TotalCount} 个沙箱工具")
#     for tool in resp.SandboxToolSet:
#         if TOOL_NAME in tool.ToolName:
#             print(f"\n找到目标工具:")
#             print(f"  名称: {tool.ToolName}")
#             print(f"  ID: {tool.ToolId}")
#             print(f"  状态: {tool.Status}")
#             tool_id = tool.ToolId
#             break
# except TencentCloudSDKException as err:
#     print(f"❌ 查询失败: {err}")

def list_sandbox_tools(tool_id=None, limit=1, offset=0):
    """
    查询沙箱工具列表
    
    Args:
        limit (int): 返回数量限制（最大100）
        offset (int): 偏移量
    
    Returns:
        response: 查询结果
    """
    try:
        # 创建请求对象
        req = models.DescribeSandboxToolListRequest()
        req.ToolIds = [tool_id]
        
        # 设置分页参数
        req.Limit = limit
        req.Offset = offset
        
        resp = client.DescribeSandboxToolList(req)
        
        print(f"📋 沙箱工具列表查询成功")
        print(f"📊 限制数量: {limit}，偏移量: {offset}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 查询失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None


print("\n=== 查询自定义类型沙箱工具 ===")
custom_tools = list_sandbox_tools(tool_id=tool_id)



=== 查询自定义类型沙箱工具 ===
📋 沙箱工具列表查询成功
📊 限制数量: 1，偏移量: 0
📋 响应: {"SandboxToolSet": [{"ToolId": "sdt-1fitksfj", "ToolName": "swe-bench-astropy-12907", "ToolType": "custom", "Status": "ACTIVE", "Description": "SWE-Bench environment with envd", "DefaultTimeoutSeconds": 900, "NetworkConfiguration": {"NetworkMode": "PUBLIC", "VpcConfig": null}, "Tags": [{"Key": "qcs:tag:createdBy", "Value": "CAMUser:100042847298:pengdrumli"}], "CreateTime": "2026-02-05T15:58:26+08:00", "UpdateTime": "2026-02-05T15:58:26+08:00", "RoleArn": "qcs::cam::uin/3321337994:roleName/tcr-full-ags", "StorageMounts": [{"Name": "envd-storage", "StorageSource": {"Cos": null, "Image": {"Reference": "ccr.ccs.tencentyun.com/archerlliu/envd:20260115_201017", "ImageRegistryType": "personal", "SubPath": "/usr/bin/envd", "Digest": "sha256:0bc7cd3b899f2f2bbf142dd11efdeee76bc95304f5b6510822f362b8a330f33e"}}, "MountPath": "/mnt/envd", "ReadOnly": true}], "CustomConfiguration": {"Image": "ccr.ccs.tencentyun.com/pengdrumli/slimshetty-sweben

创建沙箱实例

In [ ]:
# 配置 E2B
os.environ["E2B_DOMAIN"] = "ap-guangzhou.tencentags.com"
os.environ["E2B_API_KEY"] = "xxx"

print(f"✅ E2B配置完成")
print(f"🌐 Domain: {os.getenv('E2B_DOMAIN')}")

✅ E2B配置完成
🌐 Domain: ap-guangzhou.tencentags.com


In [66]:
# 创建沙箱实例
TOOL_NAME = "swe-bench-astropy-12907"
print(f"🚀 正在创建沙箱实例，工具: {TOOL_NAME}")
sandbox = Sandbox.create(template=TOOL_NAME, timeout=600)
print(f"✅ 沙箱创建成功")
print(f"📋 Sandbox ID: {sandbox.sandbox_id}")

🚀 正在创建沙箱实例，工具: swe-bench-astropy-12907
✅ 沙箱创建成功
📋 Sandbox ID: df8abcdf5b56e12344b25f0bcc9026f0a7420425


执行命令

In [67]:
# 1. 检查系统信息
print("1️⃣ 系统信息:")
result = sandbox.commands.run(cmd="uname -a", user="root")
print(result.stdout)

# 2. 检查 Python 版本
print("\n2️⃣ Python版本:")
result = sandbox.commands.run(cmd="python --version", user="root")
print(result.stdout)

# 3. 检查 envd 挂载
print("\n3️⃣ envd挂载:")
result = sandbox.commands.run(cmd="ls -la /mnt/envd | head -10", user="root")
print(result.stdout)

1️⃣ 系统信息:
Linux db7fee42 5.4.119-19-0009-03_2.0.1-00030-gfd357c61b1db #1 SMP Fri Jan 30 15:36:58 CST 2026 x86_64 x86_64 x86_64 GNU/Linux


2️⃣ Python版本:
Python 3.9.21


3️⃣ envd挂载:
-rwxr-xr-x 1 root root 13695086 Jan 15 12:02 /mnt/envd



上传文件

In [68]:
import tempfile

# 创建测试文件
print("📝 创建并上传测试文件")
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.py') as f:
    f.write("print('Hello from SWE-Bench!')\n")
    f.write("print('envd is mounted at /mnt/envd')\n")
    temp_path = f.name

# 上传文件
with open(temp_path, "r") as f:
    sandbox.files.write("test_script.py", f, user="root")
print("✅ 文件已上传")

# 执行文件
print("\n🚀 执行上传的脚本:")
result = sandbox.commands.run(cmd="python test_script.py", user="root")
print(result.stdout)

# 清理
import os as local_os
local_os.unlink(temp_path)

📝 创建并上传测试文件
✅ 文件已上传

🚀 执行上传的脚本:
Hello from SWE-Bench!
envd is mounted at /mnt/envd



执行swe-bench测试

In [70]:
print("\n执行swe test")
result = sandbox.commands.run(cmd="bash /run_tests.sh", user="root")
print(result.stdout)


执行swe test
On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   pyproject.toml

no changes added to commit (use "git add" and/or "git commit -a")
commit d16bfe05a744909de4b27f5875fe0d4ed41ce607
Merge: a4f25a2ced 95f3d4da59
Author: William Jamieson <wjamieson@stsci.edu>
Date:   Thu Mar 3 13:21:56 2022 -0500

    Merge pull request #12900 from Cadair/custom_compound_model
    
    Allow a model to override calculation of it's separability matrix

diff --git a/pyproject.toml b/pyproject.toml
index 3364d30740..02dddbe713 100644
--- a/pyproject.toml
+++ b/pyproject.toml
@@ -1,5 +1,5 @@
 [build-system]
-requires = ["setuptools",
+requires = ["setuptools==68.0.0",
             "setuptools_scm>=6.2",
             "wheel",
             "cython==0.29.22",
Using pip 25.0 from /opt/miniconda3/envs/testbed/lib/python3.9/site-packages/pip (python 3.9)
Obtaining 

清理沙箱

In [71]:
# 关闭沙箱
print("🧹 清理沙箱...")
sandbox.kill()
print("✅ 沙箱已关闭")

🧹 清理沙箱...
✅ 沙箱已关闭


删除沙箱工具

In [57]:
def delete_sandbox_tool(tool_id=None):
    """
    删除沙箱工具
    
    Args:
        tool_id (str): 工具ID
    
    Returns:
        response: 删除结果
    """
    try:
        # 创建请求对象
        req = models.DeleteSandboxToolRequest()
        
        # 设置工具标识
        if tool_id:
            req.ToolId = tool_id
        
        # 发送删除请求
        resp = client.DeleteSandboxTool(req)
        
        print(f"✅ 沙箱工具删除成功")
        if tool_id:
            print(f"🆔 已删除工具ID: {tool_id}")
        print(f"📋 响应: {resp.to_json_string()}")
        
        return resp
        
    except TencentCloudSDKException as err:
        print(f"❌ 删除失败: {err}")
        return None
    except Exception as e:
        print(f"❌ 未知错误: {e}")
        return None

# 示例：删除工具（谨慎操作）
delete_result = delete_sandbox_tool(tool_id)

✅ 沙箱工具删除成功
🆔 已删除工具ID: sdt-e21f98r1
📋 响应: {"RequestId": "b788397a-bfe8-4395-829d-9963b1486f45"}


## 总结

本 Notebook 演示了:

1. ✅ 创建带 envd 挂载的 SWE-Bench 沙箱工具
2. ✅ 使用 e2b 接口创建和管理沙箱实例
3. ✅ 在沙箱中执行命令和 Python 代码
4. ✅ 文件上传和执行
5. ✅ 验证 SWE-Bench 环境

### 关键配置

- **基础镜像**: SWE-Bench astropy 环境
- **挂载镜像**: envd 工具链
- **资源**: 4 CPU, 8Gi内存
- **网络**: PUBLIC 模式

### 下一步

- 可以基于此环境进行 SWE-Bench 评测
- 可以修改镜像以适应不同的测试任务
- 可以调整资源配置以优化性能